This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks import *


In [2]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    "embedding_corruption_ratio" : 0.1, 
    "partial_embedding_reconstruction" : True,
    'n_parts': 10
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = [GraphPartition]

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [4]:
from torch_geometric.utils.convert import to_scipy_sparse_matrix
to_scipy_sparse_matrix(dataset.edge_index).shape

(2708, 2708)

In [5]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


([4.258224010467529,
  4.23729944229126,
  4.211711883544922,
  4.181057453155518,
  4.127908706665039,
  4.075465202331543,
  4.018800735473633,
  3.915891170501709,
  3.8260979652404785,
  3.730095863342285,
  3.588977336883545,
  3.5155224800109863,
  3.348867177963257,
  3.319425582885742,
  3.25412654876709,
  3.170222520828247,
  2.9331650733947754,
  2.8733530044555664,
  2.843259811401367,
  2.819999933242798,
  2.680835485458374,
  2.603970766067505,
  2.5460948944091797,
  2.3268814086914062,
  2.4435184001922607,
  2.2844398021698,
  2.4464595317840576,
  2.362760305404663,
  2.2298054695129395,
  2.185845375061035,
  2.231945037841797,
  2.1096572875976562,
  2.038662910461426,
  2.0659964084625244,
  1.9811115264892578,
  1.9729278087615967,
  1.7731144428253174,
  1.8324685096740723,
  1.9161958694458008,
  1.8205245733261108,
  1.7816407680511475,
  1.7576696872711182,
  1.805443525314331,
  1.7843736410140991,
  1.6408953666687012,
  1.6870307922363281,
  1.673590183258